## Predicción: aprendizaje supervisado con selección de atributos FOCI

In [1]:
# Parametros de configuracion del script

dataset_name = 'GSE21050'                         # nombre del dataset
not_genes_columns = ['group','time_survivor']                           # columnas que no miden valores genicos
dataset_files_folder_preprocessing ="P1_ficheros_preprocesamiento_"+dataset_name+"/" # directorio con ficheros de preprocesamiento
dataset_files_folder_comparative ="P2_ficheros_comparativaSupervisado_"+dataset_name+"/" # directorio con ficheros de comparativa con/sin seleccion de atributos
file_source_trainval = dataset_files_folder_comparative+dataset_name+'_7_trainval.csv'    # directorio con datos de train-validation
n_splits = 5                                             # numero de particiones para validacion cruzada
n_cores = 7                                              # numero de nucleos de paralelización para FOCI                         

#### Importaciones

In [2]:
# Python imports
# ----------------------------------------

# Variable export
import pickle

# Data structure
import pandas as pd
import numpy as np
from itertools import product
from sklearn.base import clone

# Sklearn preprocess, split, pipeline, GridSearch
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import statistics

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

# Sklearn classification models
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier

from sklearn.svm import SVC
from sklearn import svm
from sklearn.svm import NuSVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import RandomForestClassifier

# Deep learning
# import tensorflow as tf
# from tensorflow import keras

# Hacer reproducibles los experimentos en sklearn y tensorflo
SEED=42
np.random.seed(SEED)

# tf.keras.utils.set_random_seed(SEED)

In [3]:
# R imports
# ---------------------------------------------

from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages
import rpy2.robjects.packages as roptions
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

# import R's "base" package
base = importr('base')

# import R's "utils" package
utils = importr('utils')

# Establecer el mirror de CRAN en el mirror de Cloud R
utils.chooseCRANmirror(ind=1)

# Instalar paquetes
# utils.install_packages('FOCI')
foci = rpackages.importr('FOCI')

# Fijar semilla en R
ro.r('set.seed({})'.format(SEED))

<rpy2.rinterface_lib.sexp.NULLType object at 0x000002130D031540> [RTYPES.NILSXP]

In [4]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

#### Lectura del dataset

In [5]:
# Division del conjunto original en un conjunto de entrenamiento-validacion y otro de test
import subprocess
subprocess.call(['python', '../split_dataset.py', 
                dataset_files_folder_preprocessing+dataset_name+'_6_diffexp.csv', 
                dataset_files_folder_comparative+dataset_name+'_7_trainval.csv',  
                dataset_files_folder_comparative+dataset_name+'_7_test.csv', 
                'group'])

0

In [6]:
dataset = pd.read_csv(file_source_trainval, sep=',', header=0, index_col=0)
dataset

,group,time_survivor,AARSD1,ABCA8,ABHD14A,ABLIM1,ACAA2,ACADSB,ACADVL,ACAN,...,ZNF367,ZNF423,ZNF436,ZNF446,ZNF503,ZNF514,ZNF700,ZNF805,ZNF93,ZWINT
162,1,1.12,6.190022,4.169925,7.065005,6.210038,6.130056,7.400026,9.439997,3.879706,...,7.750003,6.099926,6.570067,5.560104,7.360013,7.499968,6.459923,5.994918,5.715021,9.929998
264,0,2.37,7.840023,8.270015,7.380029,8.780015,6.099943,4.620000,8.830008,4.580145,...,10.569998,6.210038,7.120016,5.580145,7.290019,5.215211,6.950002,5.514903,6.205072,11.340000
187,1,0.51,5.839960,12.070000,6.295071,10.350000,7.049962,6.349967,9.420002,5.870118,...,9.410006,7.020035,8.150016,6.450056,7.829976,5.874957,7.609991,7.200023,5.064949,7.869995
35,1,2.18,7.910013,3.459432,6.169925,7.840023,6.639975,6.759955,10.100005,4.230357,...,10.960002,4.869871,5.089857,5.560104,7.609991,8.354990,9.090007,5.059998,5.829888,9.869995
61,1,2.76,7.180009,8.239980,5.955040,9.840007,5.039956,5.839960,10.110000,9.720005,...,10.499995,3.539779,5.035035,5.169925,8.250014,7.340024,9.059993,6.079957,7.980016,9.199991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,1,0.94,7.900021,7.290019,5.744957,9.140012,6.880089,5.499846,9.539992,6.450056,...,10.469998,7.199967,6.435020,5.599913,7.950002,7.600030,10.170000,6.500010,9.699994,10.100005
232,0,0.61,6.820051,8.429992,6.075019,7.460005,7.525016,7.009997,9.840007,4.189825,...,8.180009,6.380071,5.734898,5.720005,8.439997,5.814972,7.210038,7.460014,3.900062,10.189997
304,0,3.19,6.190022,10.610001,7.129983,10.860001,6.954973,6.680043,9.329998,3.959770,...,5.960002,10.939998,7.540040,5.569856,10.289996,6.755010,6.720005,6.570036,6.860019,8.350011
198,1,0.32,5.200065,2.611172,6.775034,5.809929,7.660077,4.730096,9.420002,4.760221,...,8.669983,6.190022,4.740023,6.380071,8.669983,6.234891,6.729961,6.440009,5.400175,9.639992


In [7]:
X = dataset.drop(not_genes_columns, axis=1)
X

,AARSD1,ABCA8,ABHD14A,ABLIM1,ACAA2,ACADSB,ACADVL,ACAN,ACAT2,ACOT9,...,ZNF367,ZNF423,ZNF436,ZNF446,ZNF503,ZNF514,ZNF700,ZNF805,ZNF93,ZWINT
162,6.190022,4.169925,7.065005,6.210038,6.130056,7.400026,9.439997,3.879706,9.790006,9.620000,...,7.750003,6.099926,6.570067,5.560104,7.360013,7.499968,6.459923,5.994918,5.715021,9.929998
264,7.840023,8.270015,7.380029,8.780015,6.099943,4.620000,8.830008,4.580145,8.729995,9.740000,...,10.569998,6.210038,7.120016,5.580145,7.290019,5.215211,6.950002,5.514903,6.205072,11.340000
187,5.839960,12.070000,6.295071,10.350000,7.049962,6.349967,9.420002,5.870118,7.939991,10.049998,...,9.410006,7.020035,8.150016,6.450056,7.829976,5.874957,7.609991,7.200023,5.064949,7.869995
35,7.910013,3.459432,6.169925,7.840023,6.639975,6.759955,10.100005,4.230357,7.969991,9.620000,...,10.960002,4.869871,5.089857,5.560104,7.609991,8.354990,9.090007,5.059998,5.829888,9.869995
61,7.180009,8.239980,5.955040,9.840007,5.039956,5.839960,10.110000,9.720005,9.380006,9.529997,...,10.499995,3.539779,5.035035,5.169925,8.250014,7.340024,9.059993,6.079957,7.980016,9.199991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,7.900021,7.290019,5.744957,9.140012,6.880089,5.499846,9.539992,6.450056,10.730003,9.549996,...,10.469998,7.199967,6.435020,5.599913,7.950002,7.600030,10.170000,6.500010,9.699994,10.100005
232,6.820051,8.429992,6.075019,7.460005,7.525016,7.009997,9.840007,4.189825,7.449974,9.410006,...,8.180009,6.380071,5.734898,5.720005,8.439997,5.814972,7.210038,7.460014,3.900062,10.189997
304,6.190022,10.610001,7.129983,10.860001,6.954973,6.680043,9.329998,3.959770,7.069960,10.319999,...,5.960002,10.939998,7.540040,5.569856,10.289996,6.755010,6.720005,6.570036,6.860019,8.350011
198,5.200065,2.611172,6.775034,5.809929,7.660077,4.730096,9.420002,4.760221,9.090007,10.659996,...,8.669983,6.190022,4.740023,6.380071,8.669983,6.234891,6.729961,6.440009,5.400175,9.639992


In [8]:
y = dataset['group']
y

162    1
264    0
187    1
35     1
61     1
      ..
176    1
232    0
304    0
198    1
290    0
Name: group, Length: 262, dtype: int64

#### Creación de un wrapper de FOCI para Python

Para utilizar FOCI desde sklearn (Python) debemos crear un wrapper estimador con interfaz compatible con sklearn.

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class FociSelection(BaseEstimator, TransformerMixin): 

    def __init__(self, n_features, n_cores):
        self.n_features = n_features
        self.n_cores = n_cores
        return None
    
    def fit(self, X, y = None):
        print("Fit FOCI", X.shape)

        with (ro.default_converter + pandas2ri.converter).context():
            X_r = ro.conversion.get_conversion().py2rpy(X)
            y_r = ro.conversion.get_conversion().py2rpy(pd.DataFrame(y))

        ro.r('set.seed({})'.format(SEED))        
        res_train = foci.foci(y_r,X_r,stop=False, numCores=self.n_cores, 
                               num_features=self.n_features)
        
        selected_vars_train = list(res_train.rx2('selectedVar').rx2('names'))

        # print("Selected features: ", selected_vars_train)
        # print("Selected features: ", res_train.rx2('selectedVar'))
        self.caracteristicas = selected_vars_train
        return self
    
    def transform(self, X, y = None):
        # print("Transform FOCI", X.shape)
        # print(X[self.caracteristicas].shape)
        return X[self.caracteristicas]
    
    def get_features(self):
        return self.caracteristicas

#### Pipeline y ajuste de parámetros

En este caso, definir un pipeline directamente y realizar ajuste de hiperparámetros con GridSearch y validación cruzada conllevaría una pérdida de eficiencia.  

Para cada iteración de la validación cruzada se utilizan k-1 conjuntos para train y 1 conjunto para validation.   
Debemos aplicar FOCI sobre estos conjuntos (fit sobre train y transform sobre train y validation), pero con realizarlo una vez por cada par (train, validation) es suficiente.  
En cambio, si definimos un pipeline y aplicamos seguidamente GridSearch la operación que realiza FOCI se estaría recalculado para todas las iteraciones de validación pero también para todas las combinaciones de parámetros. Y esto no es necesario, ya que el resultado de FOCI únicamente depende de los datos de la iteración y no de los parámetros del algoritmo.

Por tanto, se propone una optimización que consiste en aplicar selección FOCI sobre las "bolsas" (adecuadamente) y luego aplicar gridSearch sobre dichas bolsas de validación ya transformadas. Esto acelera muchísimo las pruebas y experimentación. No obstante, requiere de la implementación de esta transformación de las bolsas de train-validation y del gridSearch de forma manual.

##### Implementación de la transformación de las bolsas de validación cruzada

In [10]:
skfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED) 

In [11]:
def preprocess_skfold(skfold, X, y, n_features, n_cores):
    
    transformacion_skfold = []
    selected_caract = []
    
    for i, (tr_idx, test_idx) in enumerate(skfold.split(X,y)):
        
        # Selector: FOCI
        selector = FociSelection(n_features=n_features, n_cores=n_cores)
        selector.fit(X.iloc[tr_idx], y.iloc[tr_idx])
        selected_iter = selector.get_features()
        Xselected_tr_idx = selector.transform(X.iloc[tr_idx])
        Xselected_test_idx = selector.transform(X.iloc[test_idx])
        
        # Scaler: MinMaxScaler
        scaler = MinMaxScaler()
        scaler.fit(Xselected_tr_idx)
        Xscaled_tr_idx = scaler.transform(Xselected_tr_idx)
        Xscaled_test_idx = scaler.transform(Xselected_test_idx)
        
        transformacion_skfold.append(
            (Xscaled_tr_idx, y.iloc[tr_idx], 
             Xscaled_test_idx, y.iloc[test_idx])
        )
        
        selected_caract.append(selected_iter)
        
    # Guardar la ejecucion como variable en un fichero (para reutilizacion)        
    with open(dataset_files_folder_comparative+'foci_skf_'+str(n_features)+'.pkl', 'wb')  as f:
        pickle.dump((selected_caract, transformacion_skfold), f) 
        
    return selected_caract, transformacion_skfold

In [ ]:
# Transformamos las bolsas (pares train, val) de cada interacion de la validacion cruzada (solo una vez)
# FOCI = 50
skf_preprocess50 = preprocess_skfold(
    skfold=skfold, 
    X=X, y=y,
    n_features=50, 
    n_cores=n_cores)

In [ ]:
# Transformamos las bolsas (pares train, val) de cada interacion de la validacion cruzada (solo una vez)
# FOCI = 100
skf_preprocess100 = preprocess_skfold(
    skfold=skfold, 
    X=X, y=y,
    n_features=100, 
    n_cores=n_cores)

In [ ]:
# Transformamos las bolsas (pares train, val) de cada interacion de la validacion cruzada (solo una vez)
# FOCI = 150
skf_preprocess150 = preprocess_skfold(
    skfold=skfold, 
    X=X, y=y,
    n_features=150, 
    n_cores=n_cores)

##### Implementación del gridSearch sobre las bolsas ya transformadas

In [12]:
def grid_algorithm(model, param_grid, skf_preprocess):
    param_combs = list(product(*param_grid.values()))
    
    best_score = 0
    best_params = None
    
    # Averiguamos que parametros son mejores
    for params in param_combs:
        
        model.set_params(**dict(zip(param_grid.keys(), params)))
        
        # Score de validacion cruzada sobre esos parametros
        score_params_cv = []
        for (Xtrain_iter, ytrain_iter, Xtest_iter, ytest_iter) in skf_preprocess:
            model_iter = clone(model)
            model_iter.fit(Xtrain_iter, ytrain_iter)
            ypred_iter = model_iter.predict(Xtest_iter)
            score_iter = accuracy_score(ytest_iter, ypred_iter)
            score_params_cv.append(score_iter)
        
        score_params = statistics.mean(score_params_cv)
        
        # Comprobamos si esos parametros son mejores de los que ya tengo        
        if score_params > best_score:
            best_score = score_params
            best_params = dict(zip(param_grid.keys(), params))

                
    return {'model_': model.__class__.__name__, 
            'best_params_': best_params, 
            'best_score_': best_score
           }

##### Definición de pipelines y aplicación de GridSearch

In [13]:
def execute_experiments(skf_preprocess, X, y, n_features, n_cores):
    
    # Ajuste de hiperparámetros
    
    # Logistic Regression
    param_grid_LR = {
        'penalty': ['l1', 'l2'], 
        'C': [0.1, 1, 10, 20], 
        'solver': ['liblinear']}

    grid_LR = grid_algorithm(
        LogisticRegression(random_state=SEED),
        param_grid_LR,
        skf_preprocess[1]
    )
    print(grid_LR)

    with open(dataset_files_folder_comparative+'foci_grid_LR_'+str(n_features)+'.pkl', 'wb') as f:
        pickle.dump(grid_LR, f)
    
    # KNearest Neighbors
    param_grid_KNN = {
        'n_neighbors': [6,10,20,50], 
        'weights': ['uniform', 'distance'], 
        'metric': ['euclidean', 'manhattan']}

    grid_KNN = grid_algorithm(
        KNeighborsClassifier(),
        param_grid_KNN,
        skf_preprocess[1]
        )
    print(grid_KNN)

    with open(dataset_files_folder_comparative+'foci_grid_KNN_'+str(n_features)+'.pkl', 'wb') as f:
        pickle.dump(grid_KNN, f)
    
    # SVC
    param_grid_SVC = {'kernel': ['linear', 'rbf', 'sigmoid'], 
                   'C': [1, 10, 100, 1000],
                   'gamma':[1,0.1,0.001,0.0001]
                   }

    grid_SVC = grid_algorithm(
        SVC(random_state=SEED),
        param_grid_SVC,
        skf_preprocess[1]
        )
    print(grid_SVC)

    with open(dataset_files_folder_comparative+'foci_grid_SVC_'+str(n_features)+'.pkl', 'wb') as f:
        pickle.dump(grid_SVC, f)
    
    # NuSVC
    param_grid_NuSVC = {'kernel': ['linear', 'rbf', 'sigmoid'], 
                   'gamma':[1,0.1,0.001,0.0001],
                   'nu': [0.1, 0.5]  }

    grid_NuSVC = grid_algorithm(
        NuSVC(random_state=SEED),
        param_grid_NuSVC,
        skf_preprocess[1]
        )
    print(grid_NuSVC)

    with open(dataset_files_folder_comparative+'foci_grid_NuSVC_'+str(n_features)+'.pkl', 'wb') as f:
        pickle.dump(grid_NuSVC, f)
        
    # Random Forest
    param_grid_RF = {'n_estimators': [5, 10, 20, 100], 
                  'max_depth': [5, 10, 20, 50]
                  }

    grid_RF = grid_algorithm(
        RandomForestClassifier(random_state=SEED),
        param_grid_RF,
        skf_preprocess[1]
    )
    print(grid_RF)

    with open(dataset_files_folder_comparative+'foci_grid_RF_'+str(n_features)+'.pkl', 'wb') as f:
        pickle.dump(grid_RF, f)
        
    # Gradient Boosting
    param_grid_XGB = {'n_estimators': [5,10,20], 
                   'max_depth': [5, 10, 20], 
                   'learning_rate': [0.01, 0.1, 0.3, 1]
                   }

    grid_XGB = grid_algorithm(
        XGBClassifier(random_state=SEED),
        param_grid_XGB,
        skf_preprocess[1]
    )
    print(grid_XGB)

    with open(dataset_files_folder_comparative+'foci_grid_XGB_'+str(n_features)+'.pkl', 'wb') as f:
        pickle.dump(grid_XGB, f)
        
    
    # Resultados 
    df_res = pd.DataFrame(
        np.transpose([
        [grid_LR['best_params_'], grid_KNN['best_params_'], grid_SVC['best_params_'], 
         grid_NuSVC['best_params_'], grid_RF['best_params_'], 
         grid_XGB['best_params_']], 
        [grid_LR['best_score_'], grid_KNN['best_score_'], grid_SVC['best_score_'], 
            grid_NuSVC['best_score_'], grid_RF['best_score_'],
            grid_XGB['best_score_']]
        ]), 
        index=['LR', 'KNN', 'SVC', 'NuSVC', 'RF', 'XGB'], columns=['Best params', 'Best score']
    )
    print(df_res)
    
    # Almacenar los resultados en una variable externa 
    with open(dataset_files_folder_comparative+dataset_name+'_7_class_foci_'+str(n_features)+'.pkl', 'wb') as f:
        pickle.dump(df_res, f)
        
    # Guardar los resultados en un CSV        
    df_res.to_csv(dataset_files_folder_comparative+dataset_name+'_7_class_foci_'+str(n_features)+'.csv')

#### FOCI=50

In [18]:
with open(dataset_files_folder_comparative+'foci_skf_50.pkl', 'rb') as f:
    skf_preprocess50 = pickle.load(f)

In [19]:
execute_experiments(skf_preprocess50, X=X, y=y, n_features=50, n_cores=7)

{'model_': 'LogisticRegression', 'best_params_': {'penalty': 'l1', 'C': 1, 'solver': 'liblinear'}, 'best_score_': 0.7102322206095791}
{'model_': 'KNeighborsClassifier', 'best_params_': {'n_neighbors': 20, 'weights': 'uniform', 'metric': 'euclidean'}, 'best_score_': 0.6835268505079826}
{'model_': 'SVC', 'best_params_': {'kernel': 'rbf', 'C': 1, 'gamma': 1}, 'best_score_': 0.7061683599419448}
{'model_': 'NuSVC', 'best_params_': {'kernel': 'sigmoid', 'gamma': 0.0001, 'nu': 0.5}, 'best_score_': 0.7062409288824383}
{'model_': 'RandomForestClassifier', 'best_params_': {'n_estimators': 100, 'max_depth': 5}, 'best_score_': 0.7291727140783745}
{'model_': 'XGBClassifier', 'best_params_': {'n_estimators': 10, 'max_depth': 5, 'learning_rate': 0.1}, 'best_score_': 0.6986211901306241}
                                             Best params Best score
LR      {'penalty': 'l1', 'C': 1, 'solver': 'liblinear'}   0.710232
KNN    {'n_neighbors': 20, 'weights': 'uniform', 'met...   0.683527
SVC           

#### FOCI=100

In [16]:
with open(dataset_files_folder_comparative+'foci_skf_100.pkl', 'rb') as f:
    skf_preprocess100 = pickle.load(f)

In [17]:
execute_experiments(skf_preprocess100,  X=X, y=y, n_features=100, n_cores=7)

{'model_': 'LogisticRegression', 'best_params_': {'penalty': 'l2', 'C': 1, 'solver': 'liblinear'}, 'best_score_': 0.7252539912917272}
{'model_': 'KNeighborsClassifier', 'best_params_': {'n_neighbors': 50, 'weights': 'distance', 'metric': 'euclidean'}, 'best_score_': 0.6951378809869375}
{'model_': 'SVC', 'best_params_': {'kernel': 'sigmoid', 'C': 100, 'gamma': 0.001}, 'best_score_': 0.7100145137880987}
{'model_': 'NuSVC', 'best_params_': {'kernel': 'rbf', 'gamma': 0.1, 'nu': 0.5}, 'best_score_': 0.7063134978229318}
{'model_': 'RandomForestClassifier', 'best_params_': {'n_estimators': 100, 'max_depth': 5}, 'best_score_': 0.7062409288824383}
{'model_': 'XGBClassifier', 'best_params_': {'n_estimators': 20, 'max_depth': 10, 'learning_rate': 0.3}, 'best_score_': 0.6949201741654571}
                                             Best params Best score
LR      {'penalty': 'l2', 'C': 1, 'solver': 'liblinear'}   0.725254
KNN    {'n_neighbors': 50, 'weights': 'distance', 'me...   0.695138
SVC      

#### FOCI=150

In [29]:
with open(dataset_files_folder_comparative+'foci_skf_150.pkl', 'rb') as f:
    skf_preprocess150 = pickle.load(f)

In [30]:
execute_experiments(skf_preprocess150,  X=X, y=y, n_features=150, n_cores=7)

{'model_': 'LogisticRegression', 'best_params_': {'penalty': 'l2', 'C': 1, 'solver': 'liblinear'}, 'best_score_': 0.7141509433962264}
{'model_': 'KNeighborsClassifier', 'best_params_': {'n_neighbors': 50, 'weights': 'distance', 'metric': 'euclidean'}, 'best_score_': 0.7101596516690856}
{'model_': 'SVC', 'best_params_': {'kernel': 'rbf', 'C': 1, 'gamma': 0.1}, 'best_score_': 0.7100870827285921}
{'model_': 'NuSVC', 'best_params_': {'kernel': 'sigmoid', 'gamma': 0.0001, 'nu': 0.5}, 'best_score_': 0.7060957910014514}
{'model_': 'RandomForestClassifier', 'best_params_': {'n_estimators': 100, 'max_depth': 5}, 'best_score_': 0.6986211901306241}
{'model_': 'XGBClassifier', 'best_params_': {'n_estimators': 20, 'max_depth': 10, 'learning_rate': 0.3}, 'best_score_': 0.7063134978229317}
                                             Best params Best score
LR      {'penalty': 'l2', 'C': 1, 'solver': 'liblinear'}   0.714151
KNN    {'n_neighbors': 50, 'weights': 'distance', 'me...    0.71016
SVC       